> **Disclaimer**  
> This notebook uses content adapted from a course-provided notebook available at [github.com/langchain-ai/lca-langchainV1-essentials](https://github.com/langchain-ai/lca-langchainV1-essentials).  
> All original credit belongs to the course authors.

# Tools with MCP ⏰

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp-adapters`.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
load_dotenv()

from bedrock import nova_pro as llm

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Loaded 5 MCP tools: ['add_time', 'compare_time', 'convert_timezone', 'current_time', 'relative_time']


Create an agent with the MCP-provided time tools.

In [3]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model=llm,
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

Ask about the current time in San Francisco.

In [ ]:
result = await agent_with_mcp.ainvoke( #async because wea re communicating with a server
    {"messages": [{"role": "user", "content": "What's the time in SF right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What's the time in SF right now?
================================== Ai Message ==================================

[{'type': 'text', 'text': "<thinking> To find out the current time in San Francisco, I need to use the 'current_time' tool with the timezone set to 'America/Los_Angeles', which is the IANA timezone for San Francisco. </thinking>\n"}, {'type': 'tool_use', 'name': 'current_time', 'input': {'timezone': 'America/Los_Angeles'}, 'id': 'tooluse_I2RG6r96QRCYdH05pkcUwg'}]
Tool Calls:
  current_time (tooluse_I2RG6r96QRCYdH05pkcUwg)
 Call ID: tooluse_I2RG6r96QRCYdH05pkcUwg
  Args:
    timezone: America/Los_Angeles
================================= Tool Message =================================
Name: current_time

[{'type': 'text', 'text': '2026-01-04T11:40:36-08:00', 'id': 'lc_5d973d89-44af-40b3-99d8-4adf9398d3ea'}]
================================== Ai Message ==================================

The cu